In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tweetyx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
data = pd.read_csv("quora_duplicate_questions.tsv", delimiter="\t")
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [10]:
# Text preprocessing
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def expand_contractions(sentence):
    contractions = {
        "ain't": "am not",
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he'll've": "he will have",
        "he's": "he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "I'd": "I would",
        "I'd've": "I would have",
        "I'll": "I will",
        "I'll've": "I will have",
        "I'm": "I am",
        "I've": "I have",
        "isn't": "is not",
        "it'd": "it would",
        "it'd've": "it would have",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she would",
        "she'd've": "she would have",
        "she'll": "she will",
        "she'll've": "she will have",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so is",
        "that'd": "that would",
        "that'd've": "that would have",
        "that's": "that is",
        "there'd": "there would",
        "there'd've": "there would have",
        "there's": "there is",
        "they'd": "they would",
        "they'd've": "they would have",
        "they'll": "they will",
        "they'll've": "they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what'll've": "what will have",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when's": "when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who'll've": "who will have",
        "who's": "who is",
        "who've": "who have",
        "why's": "why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you would",
        "you'd've": "you would have",
        "you'll": "you will",
        "you'll've": "you will have",
        "you're": "you are",
        "you've": "you have"
    }

    # Create a regular expression pattern to match the contractions
    pattern = re.compile(r'\b(' + '|'.join(contractions.keys()) + r')\b')

    # Function to replace the matched contractions with their expanded forms
    def replace(match):
        return contractions[match.group(0)]

    # Use the sub() function with the replace function to expand contractions
    expanded_sentence = re.sub(pattern, replace, sentence)

    return expanded_sentence

def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(sentence)
    lemmatized_tokens = [lemmatizer.lemmatize(token, 'v') for token in tokens]
    lemmatized_sentence = ' '.join(lemmatized_tokens)
    return lemmatized_sentence

def delete_stopwords(sentence):
    tokens = word_tokenize(sentence)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    filtered_sentence = ' '.join(filtered_tokens)
    return filtered_sentence

def text_cleaning(x):
    # To replace consecutive whitespace and newline characters with a single space character in the string
    q = re.sub('\s+\n+', ' ', x)
    
    # To lowercase
    q = q.lower()
    
    # expand contractions
    q = expand_contractions(q)
    
    # To replace any non-alphanumeric character in the string 
    q = re.sub('[^a-zA-Z0-9]', ' ', q)
    
    # Lemmatization
    q = lemmatize_sentence(q)
    
    return delete_stopwords(q)

# Cleaning
def clean(data):
    data.drop(["id", "qid1", "qid2"], axis=1, inplace=True)
    data.dropna(axis=0, inplace=True)
    data.drop_duplicates(inplace=True)
    
    data['question1'] = data['question1'].apply(text_cleaning)
    data['question2'] = data['question2'].apply(text_cleaning)
    return data

In [11]:
import copy
data_copy = copy.deepcopy(data)
cleaned_data = clean(data_copy)
cleaned_data.tail()

,question1,question2,is_duplicate
404285,many keywords racket program language latest v...,many keywords perl program language latest ver...,0
404286,believe life death,true life death,1
404287,one coin,coin,0
404288,approx annual cost live study uic chicago indi...,little hairfall problem want use hair style pr...,0
404289,like sex cousin,like sex cousin,0


In [12]:
cleaned_data.to_csv('cleaned_data_without_stopwords.csv', index=False)

In [16]:
max = 0
l = []
for q in cleaned_data['question1'].values.astype(str):
    l.append(q.split())
    if len(q.split()) > max:
        max = len(q.split())
l[0]


['step', 'step', 'guide', 'invest', 'share', 'market', 'india']

In [ ]:
What is the step by step guide to invest in share market in india?
data['question1'].values.astype(str)

In [17]:
max

54